# Medieval Chronicles: The Fallen Knight

Welcome to this text-based RPG game set in medieval England. In this game, you'll embark on a journey with Ser Elyen, a fallen knight seeking redemption.

The game uses AI to generate dynamic responses based on your choices, alignment, and relationship with Ser Elyen. It also incorporates historical context through a Retrieval-Augmented Generation (RAG) system.

## Setup

First, let's set up the environment and initialize the necessary components:

In [ ]:
# Import helper functions
from notebook_helper import setup_notebook_environment, get_fine_tuned_prompts, get_game_environment

# Set up the environment
success = setup_notebook_environment()

if not success:
    print("Failed to set up the environment. Please check the error messages above.")
else:
    print("Environment setup successful! Ready to play the game.")


## Set API Key

If you haven't set your AI21 API key in the environment variables, you can set it here:

In [ ]:
import os

# Set your API key directly (replace with your actual API key)
# os.environ['AI21_API_KEY'] = "your_api_key_here"

# Check if API key is set
if 'AI21_API_KEY' in os.environ:
    print("API key is set.")
else:
    api_key = input("Please enter your AI21 API key: ")
    os.environ['AI21_API_KEY'] = api_key
    print("API key set successfully.")


## Initialize Game Components

Now, let's initialize the RAG system for historical context and the LLM agent for character interaction:

In [ ]:
# Initialize the RAG system and LLM agent
rag_helper, agent_helper = get_game_environment()

if rag_helper is None or agent_helper is None:
    print("Failed to initialize game components. Please check the error messages above.")
else:
    print("Game components initialized successfully!")


## Load Game Data

Let's load the game data and set up the initial game state:

In [ ]:
import json

# Load game data
with open("data/game_data.json", "r") as f:
    game_data = json.load(f)

# Initialize game state
game_state = {
    "current_scene": "intro",  # Start with the intro scene
    "player": game_data["player"],  # Copy player data from game_data
    "inventory": [],  # Start with empty inventory
    "visited_scenes": [],  # Track visited scenes
    "quest_progress": {}  # Track quest progress
}

print(f"Game loaded: {game_data['game_title']}")
print(f"Starting scene: {game_state['current_scene']}")
print(f"Player alignment: Law-Chaos: {game_state['player']['alignment']['law_chaos']}/100, Good-Evil: {game_state['player']['alignment']['good_evil']}/100")


## Game Functions

Let's define some helper functions to run the game:

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

def get_scene_by_id(scene_id):
    """Get a scene by its ID"""
    for scene in game_data["scenes"]:
        if scene["id"] == scene_id:
            return scene
    return None

def get_historical_context(scene):
    """Get historical context for a scene using the RAG system"""
    if not scene.get("historical_context_keywords"):
        return "No historical context available for this scene."
    
    # Join keywords into a query
    query = " ".join(scene["historical_context_keywords"])
    
    # Retrieve context using RAG
    context_entries = rag_helper.retrieve_context(query, num_results=2)
    
    if not context_entries:
        return "No relevant historical context found."
    
    # Format the context as a string
    context_text = ""
    for entry in context_entries:
        context_text += f"{entry['title']}: {entry['text']}\n\n"
    
    return context_text

def get_alignment_description(alignment):
    """Get a description of the player's alignment"""
    law_chaos = alignment["law_chaos"]
    good_evil = alignment["good_evil"]
    
    # Law-Chaos axis
    if law_chaos >= 75:
        law_chaos_desc = "Lawful"
    elif law_chaos >= 50:
        law_chaos_desc = "Neutral (Lawful-leaning)"
    elif law_chaos >= 25:
        law_chaos_desc = "Neutral (Chaotic-leaning)"
    else:
        law_chaos_desc = "Chaotic"
    
    # Good-Evil axis
    if good_evil >= 75:
        good_evil_desc = "Good"
    elif good_evil >= 50:
        good_evil_desc = "Neutral (Good-leaning)"
    elif good_evil >= 25:
        good_evil_desc = "Neutral (Evil-leaning)"
    else:
        good_evil_desc = "Evil"
    
    return f"{law_chaos_desc} {good_evil_desc}"

def update_alignment(alignment, action_type):
    """Update player alignment based on action type"""
    # Simple alignment adjustment based on action index
    # In a real game, this would be more sophisticated
    if action_type == 0:  # Lawful Good
        alignment["law_chaos"] = min(100, alignment["law_chaos"] + 5)
        alignment["good_evil"] = min(100, alignment["good_evil"] + 5)
    elif action_type == 1:  # Lawful Evil
        alignment["law_chaos"] = min(100, alignment["law_chaos"] + 5)
        alignment["good_evil"] = max(0, alignment["good_evil"] - 5)
    elif action_type == 2:  # Chaotic Good
        alignment["law_chaos"] = max(0, alignment["law_chaos"] - 5)
        alignment["good_evil"] = min(100, alignment["good_evil"] + 5)
    elif action_type == 3:  # Chaotic Evil
        alignment["law_chaos"] = max(0, alignment["law_chaos"] - 5)
        alignment["good_evil"] = max(0, alignment["good_evil"] - 5)
    
    return alignment

def update_relationship(relationship, action_type, alignment):
    """Update relationship with Ser Elyen based on action and alignment"""
    # Ser Elyen is Lawful Good, so he approves of actions that align with that
    trust_change = 0
    
    # Action type influence
    if action_type == 0:  # Lawful Good
        trust_change += 5
    elif action_type == 1:  # Lawful Evil
        trust_change += 0  # Approves of lawfulness but not evil
    elif action_type == 2:  # Chaotic Good
        trust_change += 0  # Approves of goodness but not chaos
    elif action_type == 3:  # Chaotic Evil
        trust_change -= 5
    
    # Alignment influence (smaller effect)
    law_chaos = alignment["law_chaos"]
    good_evil = alignment["good_evil"]
    
    # Ser Elyen values goodness more than lawfulness
    alignment_factor = (law_chaos / 100 * 0.4) + (good_evil / 100 * 0.6)  # Weighted average
    trust_change += (alignment_factor - 0.5) * 2  # Small adjustment based on overall alignment
    
    # Update trust
    relationship["trust"] = max(0, min(100, relationship["trust"] + trust_change))
    
    # Update relationship description based on trust level
    if relationship["trust"] >= 80:
        relationship["description"] = "Trusted companion"
    elif relationship["trust"] >= 60:
        relationship["description"] = "Reliable ally"
    elif relationship["trust"] >= 40:
        relationship["description"] = "Cautious acquaintance"
    elif relationship["trust"] >= 20:
        relationship["description"] = "Wary associate"
    else:
        relationship["description"] = "Distrustful stranger"
    
    return relationship

def generate_knight_response(scene, action, action_type):
    """Generate a response from Ser Elyen based on the player's action"""
    # Get historical context
    historical_context = get_historical_context(scene)
    
    # Get player alignment and relationship info
    alignment = game_state["player"]["alignment"]
    alignment_description = get_alignment_description(alignment)
    relationship = game_state["player"]["relationships"]["ser_elyen"]
    
    # Generate response from the LLM agent
    response = agent_helper.generate_agent_response(
        scene_title=scene["title"],
        action=action,
        alignment_description=alignment_description,
        law_chaos=alignment["law_chaos"],
        good_evil=alignment["good_evil"],
        relationship_description=relationship["description"],
        trust=relationship["trust"],
        historical_context=historical_context
    )
    
    # Update player alignment and relationship
    game_state["player"]["alignment"] = update_alignment(alignment, action_type)
    game_state["player"]["relationships"]["ser_elyen"] = update_relationship(
        relationship, action_type, game_state["player"]["alignment"]
    )
    
    return response

def generate_action_choices(scene):
    """Generate action choices for the player"""
    return agent_helper.generate_action_choices(
        scene_title=scene["title"],
        scene_description=scene["description"]
    )

def display_scene(scene_id):
    """Display a scene and action choices"""
    scene = get_scene_by_id(scene_id)
    if not scene:
        print(f"Error: Scene {scene_id} not found.")
        return
    
    # Add scene to visited scenes
    if scene_id not in game_state["visited_scenes"]:
        game_state["visited_scenes"].append(scene_id)
    
    # Clear previous output
    clear_output(wait=True)
    
    # Display scene title and description
    display(HTML(f"<h2>{scene['title']}</h2>"))
    display(HTML(f"<p>{scene['description']}</p>"))
    
    # Generate action choices
    action_choices = generate_action_choices(scene)
    
    # Create buttons for each action
    buttons = []
    for i, action in enumerate(action_choices):
        button = widgets.Button(
            description=action,
            layout=widgets.Layout(width='100%', height='auto')
        )
        
        # Define button click handler
        def on_button_click(b, action=action, action_type=i):
            # Generate knight's response
            response = generate_knight_response(scene, action, action_type)
            
            # Clear previous output
            clear_output(wait=True)
            
            # Display scene title and description
            display(HTML(f"<h2>{scene['title']}</h2>"))
            display(HTML(f"<p>{scene['description']}</p>"))
            
            # Display player's action
            display(HTML(f"<p><strong>You decide to:</strong> {action}</p>"))
            
            # Display knight's response
            display(HTML(f"<p><strong>Ser Elyen:</strong> "{response}"</p>"))
            
            # Display player stats
            alignment = game_state["player"]["alignment"]
            relationship = game_state["player"]["relationships"]["ser_elyen"]
            display(HTML(f"<p><em>Alignment: {get_alignment_description(alignment)} (Law-Chaos: {alignment['law_chaos']}/100, Good-Evil: {alignment['good_evil']}/100)</em></p>"))
            display(HTML(f"<p><em>Relationship with Ser Elyen: {relationship['description']} (Trust: {relationship['trust']}/100)</em></p>"))
            
            # Display continue button
            if hasattr(scene, 'next_scenes') and scene['next_scenes']:
                next_scene_buttons = []
                for next_scene_id in scene['next_scenes']:
                    next_scene = get_scene_by_id(next_scene_id)
                    if next_scene:
                        next_button = widgets.Button(
                            description=f"Go to {next_scene['title']}",
                            layout=widgets.Layout(width='100%', height='auto')
                        )
                        
                        def on_next_button_click(b, next_id=next_scene_id):
                            game_state["current_scene"] = next_id
                            display_scene(next_id)
                        
                        next_button.on_click(on_next_button_click)
                        next_scene_buttons.append(next_button)
                
                display(HTML("<h3>Where to next?</h3>"))
                for btn in next_scene_buttons:
                    display(btn)
            else:
                display(HTML("<p><em>End of current story path.</em></p>"))
        
        button.on_click(lambda b, action=action, i=i: on_button_click(b, action, i))
        buttons.append(button)
    
    # Display action buttons
    display(HTML("<h3>What will you do?</h3>"))
    for button in buttons:
        display(button)


## Start the Game

Now let's start the game with the intro scene:

In [ ]:
# Start the game with the intro scene
display_scene(game_state["current_scene"])


## Save Game State

You can save your game state to continue later:

In [ ]:
def save_game():
    """Save the current game state to a file"""
    save_data = {
        "timestamp": str(datetime.datetime.now()),
        "game_state": game_state
    }
    
    # Create saves directory if it doesn't exist
    os.makedirs("saves", exist_ok=True)
    
    # Save to file
    save_file = f"saves/save_{int(time.time())}.json"
    with open(save_file, "w") as f:
        json.dump(save_data, f, indent=2)
    
    print(f"Game saved to {save_file}")
    return save_file

def load_game(save_file):
    """Load a saved game state from a file"""
    global game_state
    
    try:
        with open(save_file, "r") as f:
            save_data = json.load(f)
        
        game_state = save_data["game_state"]
        print(f"Game loaded from {save_file}")
        print(f"Save timestamp: {save_data['timestamp']}")
        
        # Display the current scene
        display_scene(game_state["current_scene"])
        
        return True
    except Exception as e:
        print(f"Error loading game: {e}")
        return False

# Uncomment to save your game
# import datetime
# import time
# save_file = save_game()

# Uncomment to load a saved game
# load_game("saves/your_save_file.json")
